In [1]:
using CUDA
using TypedTables
using DataFrames
using Statistics
using CSV

In [2]:
# CUDA

function operation_cuda(A, B)
    C = CUDA.zeros(Float16, size(A))
    CUDA.@sync C .= A .+ B
    return C
end

function simulate_gpu(A_normal, B_normal)
    # Convert your Float64 matrices to Float32 and then to CuArray{Float32}
    A = CUDA.convert(CuArray{Float16}, A_normal)
    B = CUDA.convert(CuArray{Float16}, B_normal)
    # Mengukur waktu eksekusi
    cuda_start = CUDA.@elapsed result_gpu = operation_cuda(A, B)
    return cuda_start
end

simulate_gpu (generic function with 1 method)

In [3]:
# CPU

function operation_cpu(A, B)
    C = zeros(Float16, size(A))
    C .= A .+ B
    return C
end

function simulate_cpu(A_normal, B_normal)
    normal_start = @elapsed result_cpu = operation_cpu(A_normal, B_normal)
    return normal_start
end

simulate_cpu (generic function with 1 method)

In [4]:
function simulate(N::Int64, n_loop::Int64)
    println("Total size: $(N*N)")
    results_gpu = []
    results_cpu = []
    is_better_gpu = []
    for i in 1:n_loop
        A_normal = rand(Float16, N, N)
        B_normal = rand(Float16, N, N)
        benchmark_gpu = simulate_gpu(A_normal, B_normal)
        benchmark_cpu = simulate_cpu(A_normal, B_normal)
        push!(results_gpu, benchmark_gpu)
        push!(results_cpu, benchmark_cpu)
        push!(is_better_gpu, benchmark_gpu < benchmark_cpu)
    end
    println("Mean CPU: $(mean(results_cpu))")
    println("Mean GPU: $(mean(results_gpu))")
    return DataFrame(CPU=results_cpu, GPU=results_gpu, Better_GPU=is_better_gpu)
end

simulate (generic function with 1 method)

In [6]:
df = simulate(10, 10)
CSV.write("./1.csv", df)
df

Total size: 100
Mean CPU: 4.896999999999999e-7
Mean GPU: 0.94942695


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,1.765e-6,9.49412,false
2,4.59e-7,2.4576e-5,false
3,4.41e-7,1.7408e-5,false
4,3.2e-7,1.536e-5,false
5,2.6e-7,1.6384e-5,false
6,3.42e-7,1.536e-5,false
7,3.35e-7,1.4336e-5,false
8,3.2e-7,1.4336e-5,false
9,2.76e-7,1.4336e-5,false


In [23]:
df = simulate(50, 10)
CSV.write("./2.csv", df)
df

Total size: 2500
Mean CPU: 2.2571e-6
Mean GPU: 2.23232e-5


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,3.675e-6,4.5056e-5,false
2,2.193e-6,2.2528e-5,false
3,2.016e-6,1.9456e-5,false
4,2.27e-6,2.56e-5,false
5,2.032e-6,1.8432e-5,false
6,2.108e-6,1.6384e-5,false
7,2.036e-6,1.8432e-5,false
8,2.168e-6,1.8432e-5,false
9,1.964e-6,1.9456e-5,false


In [ ]:
df = simulate(100, 10)
CSV.write("./3.csv", df)
df

Total size: 10000
Mean CPU: 0.0023815273
Mean GPU: 0.0005711424


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.000240995,0.000602752,false
2,0.000200691,0.000589504,false
3,0.011989,0.000547456,true
4,0.00201261,0.000596192,true
5,0.00024286,0.00054912,false
6,0.000233224,0.000545536,false
7,0.00021233,0.000524576,false
8,0.00820464,0.00052816,true
9,0.000261372,0.00067952,false


In [25]:
df = simulate(500, 10)
CSV.write("./4.csv", df)
df

Total size: 250000
Mean CPU: 0.0004229625999999999
Mean GPU: 0.00011472959


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.000266523,0.000176544,true
2,0.000363702,9.2832e-5,true
3,0.000564232,0.000219424,true
4,0.000341592,9.4496e-5,true
5,0.000607998,0.000104128,true
6,0.000129485,9.4528e-5,true
7,0.000293836,9.04e-5,true
8,9.9423e-5,8.6304e-5,true
9,0.00120903,9.2768e-5,true


In [ ]:
df = simulate(1000, 10)
CSV.write("./5.csv", df)
df

Mean CPU: 0.03808624000000001
Mean GPU: 0.034511715


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.102373,0.0338499,true
2,0.036042,0.0431727,false
3,0.0331847,0.0337169,false
4,0.0282845,0.0339092,false
5,0.0266976,0.0326739,false
6,0.0309375,0.0339364,false
7,0.0281929,0.0326613,false
8,0.0310332,0.0336127,false
9,0.0331889,0.0349366,false


In [7]:
df = simulate(5000, 10)
CSV.write("./6.csv", df)
df

Total size: 25000000
Mean CPU: 0.06828963769999999
Mean GPU: 0.00859279


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.0255337,0.00852186,true
2,0.0250889,0.00857926,true
3,0.0252369,0.0086783,true
4,0.237156,0.00861869,true
5,0.0301566,0.008596,true
6,0.0270888,0.00857837,true
7,0.0245357,0.00862134,true
8,0.0266175,0.00858384,true
9,0.0288135,0.00858598,true


In [11]:
df = simulate(10000, 10)
CSV.write("./7.csv", df)
df

Total size: 100000000
Mean CPU: 0.1029982564
Mean GPU: 0.034617256


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.115745,0.0353693,true
2,0.10413,0.0345934,true
3,0.102146,0.0345645,true
4,0.101021,0.034473,true
5,0.101615,0.0344561,true
6,0.100552,0.0346676,true
7,0.0986861,0.0340877,true
8,0.103456,0.035154,true
9,0.100334,0.0345364,true


In [9]:
df = simulate(12000, 10)
CSV.write("./8.csv", df)
df

Total size: 144000000
Mean CPU: 0.1706702263
Mean GPU: 0.05084932


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.152844,0.0507771,true
2,0.150039,0.0507752,true
3,0.143741,0.0514088,true
4,0.14522,0.0500374,true
5,0.14616,0.0511594,true
6,0.367195,0.0508206,true
7,0.153099,0.0509309,true
8,0.15398,0.0505505,true
9,0.145202,0.0515425,true


In [10]:
df = simulate(15000, 10)
CSV.write("./9.csv", df)
df

Total size: 225000000
Mean CPU: 0.2826372088
Mean GPU: 0.080456495


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.28097,0.0806427,true
2,0.44095,0.0800832,true
3,0.235266,0.0795047,true
4,0.230861,0.080595,true
5,0.228501,0.0816556,true
6,0.312136,0.0800674,true
7,0.431667,0.0803558,true
8,0.2183,0.0808327,true
9,0.221465,0.0802348,true
